In [1]:
import os
os.chdir('../../src/models/')

In [5]:
from spread_model import simulate_pandemic
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from tqdm import tqdm

In [3]:
def plot_SIR_timeline(time_series, name=None, save=False):
    plt.plot(time_series.index, time_series['susceptible'], label = 'susceptible')
    plt.plot(time_series.index, time_series['infected'], label = 'infected')
    plt.plot(time_series.index, time_series['removed'], label = 'removed')
    plt.legend()
    plt.title(f'SP Network - {name}')
    if save: plt.savefig(f'SP network Simulation- {name}.png', fromat='png', dpi = 300)
    plt.show()

In [6]:
pr_list = [
    {'neighbor':1, 'work':1, 'school':1, 'home':1},
    {'neighbor':0, 'work':1, 'school':1, 'home':1},
    {'neighbor':1, 'work':0, 'school':1, 'home':1},
    {'neighbor':1, 'work':1, 'school':0, 'home':1},
    {'neighbor':1, 'work':1, 'school':1, 'home':0},
    {'neighbor':.5, 'work':.5, 'school':.5, 'home':.5},
    {'neighbor':.1, 'work':.1, 'school':.1, 'home':.1},
    {'neighbor':.01, 'work':.01, 'school':.01, 'home':.01},
    {'neighbor':.001, 'work':.001, 'school':.001, 'home':.001},
    {'neighbor':.2, 'work':.3, 'school':.5, 'home':.6},
]

from joblib import Parallel, delayed
sims = Parallel(n_jobs=-1)(delayed(simulate_pandemic)\
               (initial_infection = .0001, p_r = pr, file_path='SP_multiGraph.gpickle') for pr in tqdm(pr_list))

100%|██████████| 10/10 [00:00<00:00, 244.34it/s]


In [12]:
fig = go.Figure()

sims_dict = {'all' : 0, 'no_neighbors': 1, 'no_work':2, 'no_schools':3, 'no_houses':4,
             '50%' : 5, '10%': 6, '1%': 7, '0.1%': 8, 'real': 9, '100%': 0
            }

for i,name in enumerate(['all', 'no_neighbors', 'no_work', 'no_schools', 'no_houses']):
    time_series = sims[i][0]
    
    fig.add_trace(go.Scatter(x=time_series.index, y=time_series['infected'],
                    mode='lines',
                    name=name))
    
fig.show()    

In [21]:
import plotly.graph_objects as go

fig = go.Figure()

for name in ['100%', '50%', '10%', '1%', '0.1%', 'real']:

    time_series = sims[sims_dict[name]][0]
    
    fig.add_trace(go.Scatter(x=time_series.index, y=time_series['infected'],
                    mode='lines',
                    name=name))

    
fig.add_trace(go.Scatter(x=test_05[0].index, y=test_05[0]['infected'],
                    mode='lines',
                    name='test_05'))
    
fig.show()    

In [20]:
test_05 = simulate_pandemic(initial_infection = .0001, p_r = {'neighbor':.02, 
                                                              'work':.03, 
                                                              'school':.05, 
                                                              'home':.07},
                                            file_path='SP_multiGraph.gpickle')